In [4]:
import numpy as np
import skimage.io as io
import skimage.segmentation
import os

Prior to running this notebook, perform adaptive cell segmentation in inForm using DAPI as a nuclear marker. Export nuclear label arrays in inForm as tiff files and move them to a folder (in_dir). This notebook is used for post-processing of nuclear label arrays from inForm to produce binary masks that can be used for evaluation and comparison with inForm with other nuclear segmentation platforms.

In [5]:
def inForm_postprocessor(in_dir, save_dir):
    '''
    Converts nuclear segmentation label masks from inForm and produces binary masks for evaluation

    Parameters
    in_dir: str
        the path to the directory containing the label masks of each field from inForm
    save_dir: str
        the path to the directory where the binary masks will be saved. Each tiff image in the input directory will
        have a corresponding binary mask in the save directory
    '''

    # iterating over all files in the input directory
    for file in os.listdir(in_dir):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # reads the label mask of each field as an array
            label_mask = io.imread(os.path.join(in_dir, file))
            # Post-processing of prediction label arrays into binary masks for evaluation. The overall effect of
            # step is that nuclei that are sparse are unchanged but nuclei that are touching are eroded by one
            # pixel where they touch other nuclei. This is necessary for binary connected component analysis during
            # F1-score calculation for segmentation evaluation.
            # Finding the pixels that are touching between any two nuclei
            boundary_bool = skimage.segmentation.find_boundaries(label_mask, connectivity=label_mask.ndim,
                                                                 mode='outer', background=0)
            # Converting these pixels to the background value in the label array
            label_mask[boundary_bool] = 0
            # Converting the label array into a binary mask of foreground (255) and background (0)
            nuclei_mask_final = np.zeros((label_mask.shape[0], label_mask.shape[1]))
            nuclei_mask_final[label_mask != 0] = 255
            nuclei_mask_final = np.uint8(nuclei_mask_final)
            # saving the binary mask in the save directory
            io.imsave(os.path.join(save_dir, file[:file.find('.')] + '.tiff'), nuclei_mask_final)
        else:
            pass
        
    return None

In [6]:
# Replace the strings for in_dir and save_dir to paths on the local machine
in_dir = '/Users/abishek/Desktop/MRL/Platforms/Code/nuclear_seg/inForm_label_masks'
save_dir = '/Users/abishek/Desktop/MRL/Platforms/Code/nuclear_seg/inForm_binary_masks'
# To create binary masks from inForm label masks
inForm_postprocessor(in_dir, save_dir)